In [76]:
import pandas as pd
import numpy as np
from extract import *

In [77]:
from clean import *

In [78]:
ADDRESS = "https://openpowerlifting.gitlab.io/opl-csv/files/openpowerlifting-latest.zip"

df = pd.read_csv(extract_dataset(ADDRESS))

/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_7788/987062614.py:3: DtypeWarning: Columns (33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extract_dataset(ADDRESS))


In [79]:
# Fix Lift data
"""Fill in null data for non-completed lifts for all attempts of all lifts"""
replace_lift_null_data(df)

"""Properly returns best lift for each respective lift"""
checks_best_lift(df)

"""Calculates total for all three lifts"""
calculate_total(df)

"""Returns boolean for new column: 'ValidComp' if non-zero total and qualified i.e. no disqualifications, not showing up, failing all lifts, etc."""
df = validate_comp(df)

/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ValidComp"] = df.apply(lambda row: True if (str.isdigit(row["Place"]) and row["TotalKg"] != 0) else False, axis=1)


In [80]:
# Fix Weight data
"""Removes rows where no WeightClass nor Bodyweight are available"""
df = drop_null_weights(df)

"""Fixes all entries where WeightClass was simply a '+', takes bodyweight and uses that as the weight class"""
fix_plus_weightclass(df)

"""Created IPF Weight Class, takes WeightClass entry and removes any plus signs, assigns them into bins according to IPF weight classes"""
df = redefine_weight_classes(df)

/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female["IPFWeightClassKg"] = pd.cut(df_female["IPFWeightClassKg"], bins = F_bins, labels = F_labels)
/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male['IPFWeightClassKg'] = pd.cut(df_male['IPFWeightClassKg'], bins = M_bins, labels = M_labels)


In [81]:
# Fix testing, metric and federation data
"""If null values False, if Yes gives True"""
assume_untested(df)

"""Fill columns with no entry with placeholder"""
fill_unknown_values(df)

"""For lifters with no parent federation, assume federation they belong to is the parent federation"""
assume_federation(df)

"""All metrics with null values, taken to be zero"""
fill_lift_scores(df)

In [93]:
def rename_entries(df):
    df["AgeClass"] = df["AgeClass"].apply(lambda row: "80+" if row == "80-999" else row)
    df["BirthYearClass"] = df["BirthYearClass"].apply(lambda row: "70+" if row == "70-999" else row)
    return df

In [98]:
def discard_empty_ages(df):
    df = df.loc[~((df["Age"].isna()) & (df["AgeClass"].isna()) & (df["BirthYearClass"].isna()))]
    df = df[~((df["Age"].notna()) & (df["AgeClass"].isna()))]
    return df

In [99]:
df = rename_entries(df)

df = discard_empty_ages(df)

/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_7788/1031723635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["AgeClass"] = df["AgeClass"].apply(lambda row: "80+" if row == "80-999" else row)
/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_7788/1031723635.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["BirthYearClass"] = df["BirthYearClass"].apply(lambda row: "70+" if row == "70-999" else row)


In [113]:
df.iloc[:,24:].info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1755176 entries, 0 to 2875722
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Best3DeadliftKg   1755176 non-null  float64
 1   TotalKg           1755176 non-null  float64
 2   Place             1755176 non-null  object 
 3   Dots              1755176 non-null  float64
 4   Wilks             1755176 non-null  float64
 5   Glossbrenner      1755176 non-null  float64
 6   Goodlift          1755176 non-null  float64
 7   Tested            1755176 non-null  bool   
 8   Country           1755176 non-null  object 
 9   State             1755176 non-null  object 
 10  Federation        1755176 non-null  object 
 11  ParentFederation  1755176 non-null  object 
 12  Date              1755176 non-null  object 
 13  MeetCountry       1755176 non-null  object 
 14  MeetState         1755176 non-null  object 
 15  MeetTown          1755176 non-null  object 
 16  

In [103]:
df = df.dropna()

In [108]:
df["AgeClass"].unique()

array(['24-34', '40-44', '18-19', '16-17', '35-39', '45-49', '70-74',
       '55-59', '20-23', '50-54', '13-15', '60-64', '65-69', '75-79',
       '80+'], dtype=object)

In [109]:
def redefine_age_classes(df):
    class_conversion = {
        "5-12": "-18", "13-15": "-18", "16-17": "-18",
        "18-19": "19-23", "20-23": "19-23",
        "24-34": "24-39", "35-39": "24-39",
        "40-44": "40-49", "45-49": "40-49",
        "50-54": "50-59", "55-59": "50-59",
        "60-64": "60-69", "65-69": "60-69",
        "70-74": "70+", "75-79": "70+", "80+": "70+"
    }
    df["IPFAgeClass"] = df["AgeClass"].apply(lambda row: class_conversion[row] if type(row) == str else "Unavailable")
    pass

In [110]:
redefine_age_classes(df)

In [112]:
df["IPFAgeClass"].unique()

array(['24-39', '40-49', '19-23', '-18', '70+', '50-59', '60-69'],
      dtype=object)